# Maps for announcement blog post

Visualization 2: The carbon intensity of consumed electricity differs from generated electricity
* Show a static carbon flow map focused on a single BA plus all directly-interconnected BAs
* Pick an hour when there is some particularly dirty electricity getting imported
* Each BA would be represented by a bubble, where the color changes based on carbon intensity, and carbon flows would be represented by colored arrows between the bubbles. 
* To illustrate the difference between produced and consumed, we might want to have a pair of bubbles for each BA - one that shows the produced CI and one that shows the consumed CI. If we do this, we probably don’t want to vary the size of each bubble based on total generation. Or maybe we could do a split bubble - the top half shows produced CI and the bottom shows consumed CI.


Visualization 3: Animating hourly and consumed emissions for the whole county
* This animation should put the previous two concepts together and show how carbon flows and how CI changes across the entire country for a single day (or a week?)
* We could also potentially have two maps side by side: one that shows annual averages in bubbles (with no carbon flow), and one that shows the animated hourly flow (to really draw the distinction between annual and hourly datasets)


### Ref for making gif: 
`https://stackoverflow.com/questions/753190/programmatically-generate-video-or-animated-gif-in-python`

In [225]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import * 
import plotly.io as pio
import os
import pandas as pd
import numpy as np
from PIL import Image
import imageio

In [226]:
%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys

sys.path.append("../../src/")

import output_data

In [227]:
ba_coords = pd.read_csv("resources/ba_coords.csv", index_col=0, dtype={"cx":np.float64, "cy":np.float64})
ba_meta = pd.read_csv("../../data/manual/ba_reference.csv", index_col=0)

In [228]:
# Note: 150+ BAs are not in ba_coords
ba_list = ba_meta[(ba_meta.ba_category != "misellaneous") & (ba_meta.us_ba) & (ba_meta.index.isin(ba_coords.index))].index
ba_list = [ba for ba in ba_list if (f"{ba}.csv" in os.listdir("../../data/results/2020/power_sector_data/hourly/us_units/"))]

In [230]:
cleaned_io = pd.read_csv("../../data/outputs/2020/eia930/eia930_elec.csv", index_col=0, parse_dates=True)
cleaned_io = cleaned_io[[c for c in cleaned_io.columns if ".ID." in c]]

In [232]:
all = []
for ba in ba_list:
    produced = pd.read_csv(f"../../data/results/2020/power_sector_data/hourly/us_units/{ba}.csv", index_col="datetime_utc", parse_dates=True, usecols=["datetime_utc","fuel_category", "net_generation_mwh", "generated_co2_rate_lb_per_mwh_for_electricity"])
    produced = produced[produced.fuel_category == "total"]
    produced = produced.drop(columns=["fuel_category"])
    
    if ba_meta.loc[ba,"ba_category"] == "generation_only":
        consumed = pd.DataFrame(index=produced.index, columns=[["consumed_co2_rate_lb_per_mwh_for_electricity"]], dtype=np.float64)
    else:
        consumed = pd.read_csv(f"../../data/results/2020/carbon_accounting/hourly/us_units/{ba}.csv", index_col="datetime_utc", parse_dates=True, usecols=["datetime_utc", "consumed_co2_rate_lb_per_mwh_for_electricity"])
    consumed.columns = consumed.columns.get_level_values(0)

    both = pd.concat([produced, consumed], axis='columns')
    #both = both.loc[range_start:range_end]
    both = both.reset_index()
    both["BA"] = ba
    all.append(both)

all = pd.concat(all)

In [233]:
# Add coordinates
all = all.merge(ba_coords, how='left', validate='many_to_one', left_on="BA", right_index=True)

In [283]:
# src: https://community.plotly.com/t/how-to-include-a-colorscale-for-color-of-line-graphs/38002/3 
from ast import literal_eval
def get_color_for_val(val, vmin, vmax, pl_colors):
    if pl_colors[0][:3] != 'rgb':
        raise ValueError('This function works only with Plotly  rgb-colorscales')
    if vmin >= vmax:
        raise ValueError('vmin should be < vmax')

    scale = [round(k / (len(pl_colors)), 3) for k in range(len(pl_colors) + 1)]

    colors_01 = np.array([literal_eval(color[3:]) for color in pl_colors]) / 255  # color codes in [0,1]

    v = (val - vmin) / (vmax - vmin)  # val is mapped to v in [0,1]
    # find two consecutive values in plotly_scale such that   v is in  the corresponding interval
    idx = 0

    while idx < (len(scale)-2) and (v > scale[idx + 1]):
        idx += 1

    vv = (v - scale[idx]) / (scale[idx + 1] - scale[idx])

    # get   [0,1]-valued color code representing the rgb color corresponding to val
    if idx == len(pl_colors)-1: # Make this work when some values exceed range
        val_color01 = colors_01[idx] # color by last color 
    else: 
        val_color01 = colors_01[idx] + vv * (colors_01[idx + 1] - colors_01[idx])

    val_color_0255 = (255 * val_color01 + 0.5).astype(int)
    return f'rgb{str(tuple(val_color_0255))}'

In [ ]:
range_start = "2020-08-01T04:00+00"
range_end = "2020-08-3T04:00+00"

#range_start = "2020-07-21T12:00+00"
#range_end = "2020-07-23T12:00+00"

In [285]:
dir_name = "maps"

c_max = 2200 #np.floor(all.generated_co2_rate_lb_per_mwh_for_electricity.max() + 100)

hours = all.datetime_utc[(all.datetime_utc < pd.to_datetime(range_end)) & (all.datetime_utc > pd.to_datetime(range_start)) & (all.BA == "CISO")]
for hour in hours:
    print(hour, end="...")
    io_toplot = cleaned_io.loc[hour]
    toplot = all[all.datetime_utc == hour]
    fig = go.Figure()

    toplot.loc[toplot.net_generation_mwh < 1, "net_generation_mwh"] = 1
    sizes = np.log(toplot.net_generation_mwh)/np.log(1.5)
    offset = sizes/2.5

    colorscale = diverging.RdYlGn_r
    #colorscale = cmocean.solar_r

    ### From when 
    # max_width = io_toplot.max()
    # width_factor = 8/max_width
    #width_factor = 1/200
    for name, val in io_toplot.iteritems():
        if val <= 0: 
            continue 
        bas = name.split(".")[1].split("-")
        (ba1, ba2) = bas

        next = False
        for ba in bas: 
            if ba not in ba_coords.index:
                next=True
            if ba not in toplot.BA.unique():
                next=True
        if next:
            continue

        color = toplot.loc[toplot.BA == ba1, "generated_co2_rate_lb_per_mwh_for_electricity"].to_numpy()[0]

        fig.add_trace(
            go.Scatter(x = ba_coords.loc[bas,"cx"], y = ba_coords.loc[bas,"cy"], opacity=1.0,
                mode="lines", line = dict(color=get_color_for_val(color, 0, c_max, colorscale), width=2), showlegend=False
            )
        )

    ################################# Plot BAs 
    toplot.loc[toplot.net_generation_mwh < 1, "net_generation_mwh"] = 1
    sizes = np.log(toplot.net_generation_mwh)/np.log(1.5)
    offset = sizes/1.6

    # Zero-generation BAs: plot under BAs with non-zero gen 
    zero_gen_bas = (toplot.net_generation_mwh == 1) # (we set to 1 above to make log work)
    fig.add_trace(
        go.Scatter(x=toplot.loc[zero_gen_bas,"cx"], y=toplot.loc[zero_gen_bas,"cy"]+(2/2.5), mode="markers", 
            hoverinfo="text", text=toplot.loc[zero_gen_bas,"BA"], 
            marker_symbol="triangle-up",
            marker=dict(color='lightgrey', 
                line=dict(width=1, color='DarkSlateGrey'),
                size=7, opacity=1.0,
                sizemode='diameter'),
            showlegend=False  
        )
    )

    fig.add_trace(
        go.Scatter(x=toplot.cx, y=toplot.cy, mode="markers", hoverinfo="text", text=toplot.BA, 
            marker_symbol="triangle-up", 
            opacity=1.0,
            marker=dict(color=toplot.generated_co2_rate_lb_per_mwh_for_electricity, size=sizes,
                sizemode='diameter', cmin=0, cmax=c_max, opacity=1.0,
                line=dict(width=1, color='DarkSlateGrey'),
            colorscale="rdylgn_r"),
            name="Generated",   
            showlegend=False
        )
    )
    consumed_toplot = ~toplot.consumed_co2_rate_lb_per_mwh_for_electricity.isna()
    fig.add_trace(
        go.Scatter(x=toplot.loc[consumed_toplot,"cx"], y=toplot.loc[consumed_toplot,"cy"]+offset[consumed_toplot], mode="markers", 
            hoverinfo="text", text=toplot.loc[consumed_toplot,"BA"], 
            marker_symbol="triangle-down",
            marker=dict(color=toplot.loc[consumed_toplot,"consumed_co2_rate_lb_per_mwh_for_electricity"], 
                size=sizes[consumed_toplot], opacity=1.0,
                line=dict(width=1, color='DarkSlateGrey'),
                sizemode='diameter', cmin=0, cmax=c_max, 
                colorbar=dict(
                    title="Emission rate<br>(lbs/MWh)", orientation='v', len=.8, 
                    thickness=20, yanchor='bottom', y=0, xpad=20
                ),
                colorscale="rdylgn_r"
            ),
            name="Consumed", 
            showlegend=False  
        )
    )

    # Legends: don't want colored markers
    fig.add_trace(
        go.Scatter(x=[-10], y=[-10], mode="markers", 
            marker_symbol="triangle-up",
            marker=dict(color='white', line=dict(width=2, color='DarkSlateGrey'), size=10),
            name="Generated",   
        )
    )
    fig.add_trace(
        go.Scatter(x=[-10], y=[-10], mode="markers", 
            marker_symbol="triangle-down",
            marker=dict(color='white', line=dict(width=2, color='DarkSlateGrey'), size=10),
            name="Consumed",   
        )
    )
    fig.update_yaxes(range=(550,0)) # autorange="reversed")
    fig.update_xaxes(range=(0,800))

    # Add images
    fig.add_layout_image(
            dict(
                source=Image.open("resources/usa.png"),
                xref="x",
                yref="y",
                x=10,
                y=0,
                sizex=790,
                sizey=550,
                sizing="stretch",
                opacity=0.5,
                layer="below")
    )

    # Add images
    fig.add_layout_image(
            dict(
                source=Image.open("resources/legend_bottom_smaller.png"),
                xref="x",
                yref="y",
                x=-20,
                y=400,
                sizex=260,
                sizey=200,
                sizing="contain",
                opacity=1.0,
                layer="below")
    )

    # Set templates
    fig.update_layout(template="plotly_white", width=800, height=600, 
        yaxis_visible=False, xaxis_visible=False,
        title=hour.tz_convert("US/Eastern").strftime("%B %-d, %Y - %-I:00 %p ET"))
    #fig.show()
    os.makedirs(f"outputs/{dir_name}/", exist_ok=True)
    fig.write_image(f"outputs/{dir_name}/{hour}.png", scale=2)


2020-07-21 13:00:00+00:00...2020-07-21 14:00:00+00:00...2020-07-21 15:00:00+00:00...2020-07-21 16:00:00+00:00...2020-07-21 17:00:00+00:00...2020-07-21 18:00:00+00:00...2020-07-21 19:00:00+00:00...2020-07-21 20:00:00+00:00...2020-07-21 21:00:00+00:00...2020-07-21 22:00:00+00:00...2020-07-21 23:00:00+00:00...2020-07-22 00:00:00+00:00...2020-07-22 01:00:00+00:00...2020-07-22 02:00:00+00:00...2020-07-22 03:00:00+00:00...2020-07-22 04:00:00+00:00...2020-07-22 05:00:00+00:00...2020-07-22 06:00:00+00:00...2020-07-22 07:00:00+00:00...2020-07-22 08:00:00+00:00...2020-07-22 09:00:00+00:00...2020-07-22 10:00:00+00:00...2020-07-22 11:00:00+00:00...2020-07-22 12:00:00+00:00...2020-07-22 13:00:00+00:00...2020-07-22 14:00:00+00:00...2020-07-22 15:00:00+00:00...2020-07-22 16:00:00+00:00...2020-07-22 17:00:00+00:00...2020-07-22 18:00:00+00:00...2020-07-22 19:00:00+00:00...2020-07-22 20:00:00+00:00...2020-07-22 21:00:00+00:00...2020-07-22 22:00:00+00:00...2020-07-22 23:00:00+00:00...2020-07-23 00:00:00+

In [286]:
## Make gif 
images = []
files = [f for f in os.listdir(f"outputs/{dir_name}/") if ".png" in f]
files.sort()
for f in files:
    images.append(imageio.imread(f"outputs/{dir_name}/"+f))
imageio.mimsave(f"outputs/movie_{dir_name}.gif", images)

/var/folders/mr/3dcnnc3153j6s05bmdwr0r0h0000gq/T/ipykernel_60611/3998957241.py:6: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.



In [238]:
# Now just CISO and neighbors. 

# Get list of CISO neighbors 
ciso_interchanges = [c for c in cleaned_io.columns if "CISO" in c]
ciso_bas = []
for ci in ciso_interchanges: 
    ba1, ba2 = ci.split(".")[1].split("-")
    if ba1 not in ciso_bas: 
        ciso_bas.append(ba1)
    if ba2 not in ciso_bas:
        ciso_bas.append(ba2)



In [239]:
# Identify day with max rate difference in CISO 
tester = (all[all.BA=="CISO"]).copy()
tester["difference"] = tester.consumed_co2_rate_lb_per_mwh_for_electricity - tester.generated_co2_rate_lb_per_mwh_for_electricity
tester.difference.abs().max()
tester[tester.difference == tester.difference.abs().max()]

In [267]:
# Hour with max CISO difference generated / consumed 
hour = '2020-09-25 12:00:00+00:00'

io_toplot = cleaned_io.loc[hour, ciso_interchanges]
toplot = all[all.datetime_utc == hour]
toplot = toplot[toplot.BA.isin(ciso_bas)]
fig = go.Figure()

colorscale = diverging.RdYlGn_r
#colorscale = cmocean.solar_r

c_max = np.floor(toplot.generated_co2_rate_lb_per_mwh_for_electricity.max() + 100)

### From when 
# max_width = io_toplot.max()
# width_factor = 8/max_width
#width_factor = 1/200
for name, val in io_toplot.iteritems():
    if val <= 0: 
        continue 
    bas = name.split(".")[1].split("-")
    (ba1, ba2) = bas

    next = False
    for ba in bas: 
        if ba not in ba_coords.index:
            next=True
        if ba not in toplot.BA.unique():
            next=True
    if next:
        continue

    color = toplot.loc[toplot.BA == ba1, "generated_co2_rate_lb_per_mwh_for_electricity"].to_numpy()[0]
    print(color)

    fig.add_trace(
        go.Scatter(x = ba_coords.loc[bas,"cx"], y = ba_coords.loc[bas,"cy"], opacity=1.0,
            mode="lines", line = dict(color=get_color_for_val(color, 0, c_max, colorscale), width=2), showlegend=False
        )
    )

################################# Plot BAs 
toplot.loc[toplot.net_generation_mwh < 1, "net_generation_mwh"] = 1
sizes = np.log(toplot.net_generation_mwh)/np.log(1.5)
offset = sizes/1.6
fig.add_trace(
    go.Scatter(x=toplot.cx, y=toplot.cy-offset, mode="markers", hoverinfo="text", text=toplot.BA, 
        marker_symbol="triangle-up", 
        opacity=1.0,
        marker=dict(line=dict(width=1, color='DarkSlateGrey'),
            color=toplot.generated_co2_rate_lb_per_mwh_for_electricity, size=sizes,
            sizemode='diameter', cmin=0, cmax=c_max, opacity=1.0,
        colorscale="rdylgn_r"),
        name="Generated",   
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(x=toplot.cx, y=toplot.cy, mode="markers", hoverinfo="text", text=toplot.BA, 
        marker_symbol="triangle-down",
        marker=dict(color=toplot.consumed_co2_rate_lb_per_mwh_for_electricity, size=sizes, opacity=1.0,
            line=dict(width=1, color='DarkSlateGrey'),
            sizemode='diameter', cmin=0, cmax=c_max, colorbar=dict(
            title="Emission rate<br>(lbs/MWh)", orientation='v', len=.8, thickness=20, yanchor='bottom', y=0, xpad=20
        ),
        colorscale="rdylgn_r"),
        name="Consumed", 
        showlegend=False  
    )
)


# Legends: don't want colored markers
# Legends: don't want colored markers
fig.add_trace(
    go.Scatter(x=[-10], y=[-10], mode="markers", 
        marker_symbol="triangle-up",
        marker=dict(color='white', line=dict(width=2, color='DarkSlateGrey'), size=10),
        name="Generated",   
    )
)
fig.add_trace(
    go.Scatter(x=[-10], y=[-10], mode="markers", 
        marker_symbol="triangle-down",
        marker=dict(color='white', line=dict(width=2, color='DarkSlateGrey'), size=10),
        name="Consumed",   
    )
)
fig.update_yaxes(range=(500,0)) # autorange="reversed")
fig.update_xaxes(range=(0,200))

## loop through the labels and add them as annotations
for x in zip(toplot.BA, toplot.cx, toplot.cy):
    left_bas = ["BANC","TIDC","CISO","LDWP","IID"]
    delta = (-12 if x[0] in left_bas else 12)
    fig.add_annotation(
        x=x[1] + delta,
        y=x[2],
        text=x[0],
        showarrow=False,
        xanchor=('right' if x[0] in left_bas else 'left')
    )

# Add images
fig.add_layout_image(
        dict(
            source=Image.open("resources/usa.png"),
            xref="x",
            yref="y",
            x=10,
            y=0,
            sizex=790,
            sizey=550,
            sizing="stretch",
            opacity=0.5,
            layer="below")
)

# Add images
fig.add_layout_image(
        dict(
            source=Image.open("resources/legend_bottom_smaller.png"),
            xref="x",
            yref="y",
            x=-20,
            y=360,
            sizex=220,
            sizey=144,
            sizing="contain",
            opacity=1.0,
            layer="below")
)

# Set templates
fig.update_layout(template="plotly_white", width=400, height=550,
    yaxis_visible=False, xaxis_visible=False,
    title=pd.to_datetime(hour).tz_convert("US/Pacific").strftime("%B %-d, %Y - %-I:00 %p PT")
)
fig.show()
fig.write_image(f"outputs/viz2_legend.png", scale=3) 

2051.357942055572
791.7235362136025
494.285163780544
486.6971936773157
468.0531456179542
1171.858214304538
873.4998685340672
725.9808186972756
640.9137194410688
742.8031879262338


In [269]:
toplot["difference"] = (toplot.generated_co2_rate_lb_per_mwh_for_electricity - toplot.consumed_co2_rate_lb_per_mwh_for_electricity)/toplot.generated_co2_rate_lb_per_mwh_for_electricity
toplot

,datetime_utc,net_generation_mwh,generated_co2_rate_lb_per_mwh_for_electricity,consumed_co2_rate_lb_per_mwh_for_electricity,BA,cx,cy,difference
6437,2020-09-25 12:00:00+00:00,2529.32,2051.357942,1267.509927,AZPS,156.0,316.0,0.382112
6436,2020-09-25 12:00:00+00:00,761.04,791.723536,599.596239,BANC,36.0,204.0,0.242670
6436,2020-09-25 12:00:00+00:00,9513.34,494.285164,439.173781,BPAT,80.0,100.0,0.111497
6436,2020-09-25 12:00:00+00:00,13300.68,486.697194,623.864267,CISO,60.0,270.0,-0.281832
6436,2020-09-25 12:00:00+00:00,577.47,468.053146,475.366265,IID,92.0,334.0,-0.015625
6437,2020-09-25 12:00:00+00:00,1897.30,1171.858214,935.873974,LDWP,76.0,304.0,0.201376
6436,2020-09-25 12:00:00+00:00,3422.94,873.499869,937.618252,NEVP,96.0,210.0,-0.073404
6436,2020-09-25 12:00:00+00:00,723.10,725.980819,717.942345,PACW,56.0,130.0,0.011073
6437,2020-09-25 12:00:00+00:00,8615.73,640.913719,656.187786,SRP,172.0,344.0,-0.023832
6436,2020-09-25 12:00:00+00:00,278.83,803.234946,764.286810,TIDC,44.0,252.0,0.048489


In [278]:
all["difference"] = ((all.generated_co2_rate_lb_per_mwh_for_electricity - all.consumed_co2_rate_lb_per_mwh_for_electricity).abs())* all.net_generation_mwh
px.line(all.groupby("datetime_utc").mean()["difference"])

In [277]:
all[all.datetime_utc=="12-01-2020 T05:00+00:00"].to_csv("outputs/problem_date.csv")

In [ ]:
### Old code for arrows
    # # Arrows have to be added separately
    # line_size = val*width_factor
    # fig.add_annotation(
    #     x=ba_coords.loc[ba2,"cx"],  # arrows' head
    #     y=ba_coords.loc[ba2,"cy"],  # arrows' head
    #     ax=ba_coords.loc[ba1,"cx"],  # arrows' tail
    #     ay=ba_coords.loc[ba1,"cy"],  # arrows' tail
    #     xref='x',
    #     yref='y',
    #     axref='x',
    #     ayref='y',
    #     text='',  # if you want only the arrow
    #     showarrow=True,
    #     arrowhead=1,
    #     arrowsize=1, #max(.3, line_size),
    #     arrowwidth=1,
    #     arrowcolor='royalblue'
    # )